In [ ]:
# Model Download
!wget -O face_landmarker_v2_with_blendshapes.task -q https://storage.googleapis.com/mediapipe-models/face_landmarker/face_landmarker/float16/1/face_landmarker.task

In [ ]:
# Download packages
!pip install -r requirements.txt

In [ ]:
# Import Packages
import cv2
import mediapipe as mp
import numpy as np
import time
import sqlite3

from mediapipe.tasks import python
from mediapipe.tasks.python import vision

In [2]:
# SQL Connection
conn = sqlite3.connect("fatigue_data.sqlite")
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS fatigue_data (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    timestamp REAL,
    ear REAL,
    mar REAL,
    roll REAL,
    pitch REAL,
    label INTEGER
)
""")
conn.commit()

In [3]:
# ---------- Load Model ----------
base_options = python.BaseOptions(model_asset_path='face_landmarker_v2_with_blendshapes.task')

# Model Configuration(s)
options = vision.FaceLandmarkerOptions(
    base_options=base_options,
    running_mode=vision.RunningMode.VIDEO,
    num_faces=1,
    output_face_blendshapes=False,
    output_facial_transformation_matrixes=False
)

landmarker = vision.FaceLandmarker.create_from_options(options)


W0000 00:00:1771060890.263230  111699 face_landmarker_graph.cc:174] Sets FaceBlendshapesGraph acceleration to xnnpack by default.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1771060890.273121  111705 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1771060890.288486  111708 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [4]:
# Utility Functions
def calculate_angle(p1, p2):
    dx = p2[0] - p1[0]
    dy = p2[1] - p1[1]
    return np.degrees(np.arctan2(dy, dx))

def euclidean(p1, p2):
    return np.linalg.norm(np.array(p1) - np.array(p2))

def calculate_ear(eye_pts):
    p1, p2, p3, p4, p5, p6 = eye_pts
    return (euclidean(p2,p6) + euclidean(p3,p5)) / (2.0 * euclidean(p1,p4))

def calculate_mar(mouth_pts):
    p1, p2, p3, p4, p5, p6 = mouth_pts[:6]
    return (euclidean(p2,p6) + euclidean(p3,p5)) / (2.0 * euclidean(p1,p4))


In [5]:
# ---------------------- Landmark Indices ----------------------
LEFT_EYE = [33, 160, 158, 133, 153, 144]
RIGHT_EYE = [362, 385, 387, 263, 373, 380]
MOUTH = [13, 14, 78, 308, 82, 312, 87, 317]

In [6]:
cap = cv2.VideoCapture(0)
cv2.namedWindow("Fatigue Data Collector", cv2.WINDOW_NORMAL)

recording = False
waiting_for_label = False
recorded_data = []
record_start_time = None
message = "Press R to start recording | Q to quit"

global_start_time = time.time()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    mesh_frame = frame.copy()

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_frame)
    timestamp = int((time.time() - global_start_time) * 1000)

    result = landmarker.detect_for_video(mp_image, timestamp)

    if result.face_landmarks:
        face_landmarks = result.face_landmarks[0]
        h, w, _ = frame.shape

        nose = face_landmarks[1]
        left_eye_outer = face_landmarks[33]
        right_eye_outer = face_landmarks[263]
        chin = face_landmarks[152]

        nose_pt = (int(nose.x * w), int(nose.y * h))
        left_eye_pt = (int(left_eye_outer.x * w), int(left_eye_outer.y * h))
        right_eye_pt = (int(right_eye_outer.x * w), int(right_eye_outer.y * h))
        chin_pt = (int(chin.x * w), int(chin.y * h))

        roll_angle = calculate_angle(left_eye_pt, right_eye_pt)
        pitch_distance = chin_pt[1] - nose_pt[1]

        left_eye_pts = [(int(face_landmarks[i].x*w), int(face_landmarks[i].y*h)) for i in LEFT_EYE]
        right_eye_pts = [(int(face_landmarks[i].x*w), int(face_landmarks[i].y*h)) for i in RIGHT_EYE]
        mouth_pts = [(int(face_landmarks[i].x*w), int(face_landmarks[i].y*h)) for i in MOUTH]

        ear = (calculate_ear(left_eye_pts) + calculate_ear(right_eye_pts)) / 2.0
        mar = calculate_mar(mouth_pts)
        
                # -------- Draw Eye Landmarks (Green) --------
        for pt in left_eye_pts + right_eye_pts:
            cv2.circle(mesh_frame, pt, 3, (0, 255, 0), -1)

        # -------- Draw Mouth Landmarks (Red) --------
        for pt in mouth_pts:
            cv2.circle(mesh_frame, pt, 3, (0, 0, 255), -1)

        # -------- Draw Head Tilt Lines --------
        cv2.line(mesh_frame, left_eye_pt, right_eye_pt, (255, 0, 0), 2)
        cv2.line(mesh_frame, nose_pt, chin_pt, (255, 255, 0), 2)

        # Optional: Show feature values
        cv2.putText(mesh_frame, f"EAR: {ear:.3f}", (30, 80),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

        cv2.putText(mesh_frame, f"MAR: {mar:.3f}", (30, 110),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

        cv2.putText(mesh_frame, f"Roll: {roll_angle:.2f}", (30, 140),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)


        if recording:
            recorded_data.append((timestamp, ear, mar, roll_angle, pitch_distance))

    # ---------------------- UI STATES ----------------------

    if recording:
        elapsed = int(time.time() - record_start_time)
        remaining = 5 - elapsed

        cv2.putText(mesh_frame, f"REC ● {remaining}s", (30, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)

        if remaining <= 0:
            recording = False
            waiting_for_label = True
            message = "Press 0 = Normal | 1 = Fatigue"

    elif waiting_for_label:
        cv2.putText(mesh_frame, message, (30, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

    else:
        cv2.putText(mesh_frame, message, (30, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    combined = np.hstack((frame, mesh_frame))
    cv2.imshow("Fatigue Data Collector", combined)

    key = cv2.waitKey(1) & 0xFF

    if key == ord('r') and not recording and not waiting_for_label:
        recording = True
        record_start_time = time.time()
        recorded_data = []
        message = ""

    elif key == ord('0') and waiting_for_label:
        for row in recorded_data:
            cursor.execute("INSERT INTO fatigue_data (timestamp, ear, mar, roll, pitch, label) VALUES (?, ?, ?, ?, ?, ?)",
                           (*row, 0))
        conn.commit()
        waiting_for_label = False
        message = "Saved as NORMAL | Press R to record again"

    elif key == ord('1') and waiting_for_label:
        for row in recorded_data:
            cursor.execute("INSERT INTO fatigue_data (timestamp, ear, mar, roll, pitch, label) VALUES (?, ?, ?, ?, ?, ?)",
                           (*row, 1))
        conn.commit()
        waiting_for_label = False
        message = "Saved as FATIGUE | Press R to record again"

    elif key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
conn.close()

QFontDatabase: Cannot find font directory /home/sonu/Desktop/fatigue/.venv/lib/python3.10/site-packages/cv2/qt/fonts.
Note that Qt no longer ships fonts. Deploy some (from https://dejavu-fonts.github.io/ for example) or switch to fontconfig.
QFontDatabase: Cannot find font directory /home/sonu/Desktop/fatigue/.venv/lib/python3.10/site-packages/cv2/qt/fonts.
Note that Qt no longer ships fonts. Deploy some (from https://dejavu-fonts.github.io/ for example) or switch to fontconfig.
QFontDatabase: Cannot find font directory /home/sonu/Desktop/fatigue/.venv/lib/python3.10/site-packages/cv2/qt/fonts.
Note that Qt no longer ships fonts. Deploy some (from https://dejavu-fonts.github.io/ for example) or switch to fontconfig.
QFontDatabase: Cannot find font directory /home/sonu/Desktop/fatigue/.venv/lib/python3.10/site-packages/cv2/qt/fonts.
Note that Qt no longer ships fonts. Deploy some (from https://dejavu-fonts.github.io/ for example) or switch to fontconfig.
QFontDatabase: Cannot find font 

In [9]:
import sqlite3
import pandas as pd

# Connect to database
conn = sqlite3.connect("fatigue_data.sqlite")

# Read full table into pandas dataframe
df = pd.read_sql_query("SELECT * FROM fatigue_data", conn)

conn.close()

# Show first 10 rows
df.head(10)


,id,timestamp,ear,mar,roll,pitch,label
0,1,13041.0,0.298366,0.519745,1.181189,76.0,0
1,2,13077.0,0.294782,0.523849,1.193489,76.0,0
2,3,13109.0,0.297783,0.520574,1.193489,76.0,0
3,4,13141.0,0.297694,0.520574,0.596809,76.0,0
4,5,13177.0,0.296279,0.542947,0.596809,76.0,0
5,6,13209.0,0.296279,0.544879,0.596809,76.0,0
6,7,13241.0,0.295070,0.544726,0.596809,76.0,0
7,8,13277.0,0.295070,0.520428,0.596809,76.0,0
8,9,13309.0,0.289809,0.520428,0.590657,76.0,0
9,10,13341.0,0.281253,0.546044,0.590657,76.0,0


In [2]:
import cv2
import mediapipe as mp
import numpy as np
import time
from collections import deque
from dataclasses import dataclass
from typing import Tuple

from mediapipe.tasks import python
from mediapipe.tasks.python import vision

@dataclass
class FatigueState:
    is_fatigued: bool = False
    fatigue_level: float = 0.0  # 0-100%
    alerts: list = None
    
    def __post_init__(self):
        self.alerts = []

class SimpleFatigueDetector:
    def __init__(self):
        # Simple threshold-based rules
        self.EAR_THRESHOLD = 0.25  # Eye Aspect Ratio threshold (lower = more closed)
        self.MAR_THRESHOLD = 0.6   # Mouth Aspect Ratio threshold (higher = more yawn-like)
        self.HEAD_TILT_THRESHOLD = 15  # Degrees of head tilt
        
        # Temporal smoothing (look at last N frames)
        self.history_size = 30  # About 1 second at 30fps
        self.ear_history = deque(maxlen=self.history_size)
        self.mar_history = deque(maxlen=self.history_size)
        self.head_tilt_history = deque(maxlen=self.history_size)
        
        # Fatigue counters
        self.consecutive_closed_eyes = 0
        self.CLOSED_EYE_THRESHOLD = 15  # Frames
        
        # Alert cooldown (to avoid spam)
        self.last_alert_time = 0
        self.alert_cooldown = 5  # seconds
        
    def update_histories(self, ear, mar, head_tilt):
        self.ear_history.append(ear)
        self.mar_history.append(mar)
        self.head_tilt_history.append(head_tilt)
    
    def check_eye_closed(self, ear) -> Tuple[bool, str]:
        """Check if eyes are closed or nearly closed"""
        if ear < self.EAR_THRESHOLD:
            self.consecutive_closed_eyes += 1
            if self.consecutive_closed_eyes > self.CLOSED_EYE_THRESHOLD:
                return True, f"Eyes closed for {self.consecutive_closed_eyes} frames"
        else:
            self.consecutive_closed_eyes = max(0, self.consecutive_closed_eyes - 2)
        return False, ""
    
    def check_yawn(self, mar) -> Tuple[bool, str]:
        """Check for yawning (high MAR)"""
        if mar > self.MAR_THRESHOLD:
            # Check if it's sustained (look at recent history)
            recent_mar = list(self.mar_history)[-10:]
            if len(recent_mar) >= 5 and sum(m > self.MAR_THRESHOLD for m in recent_mar) >= 3:
                return True, "Possible yawning detected"
        return False, ""
    
    def check_head_tilt(self, head_tilt) -> Tuple[bool, str]:
        """Check for excessive head tilting (looking down/drooping)"""
        if abs(head_tilt) > self.HEAD_TILT_THRESHOLD:
            return True, f"Excessive head tilt: {head_tilt:.1f}°"
        return False, ""
    
    def check_fatigue_trend(self) -> Tuple[bool, str]:
        """Check for gradual fatigue indicators (trends)"""
        if len(self.ear_history) < self.history_size:
            return False, ""
        
        # Check if EAR is trending downward (eyes getting more closed)
        ear_list = list(self.ear_history)
        first_half_ear = np.mean(ear_list[:15])
        second_half_ear = np.mean(ear_list[15:])
        
        if second_half_ear < first_half_ear * 0.8:  # 20% decrease
            return True, "Eyes gradually closing"
        
        return False, ""
    
    def detect(self, ear, mar, head_tilt, current_time) -> FatigueState:
        """Main detection function"""
        state = FatigueState()
        
        # Update histories
        self.update_histories(ear, mar, head_tilt)
        
        # Check individual rules
        eye_closed, eye_msg = self.check_eye_closed(ear)
        yawning, yawn_msg = self.check_yawn(mar)
        head_tilted, tilt_msg = self.check_head_tilt(head_tilt)
        trend_fatigue, trend_msg = self.check_fatigue_trend()
        
        # Collect alerts (with cooldown)
        if current_time - self.last_alert_time > self.alert_cooldown:
            if eye_closed:
                state.alerts.append(("⚠️ DROWSY", eye_msg))
            if yawning:
                state.alerts.append(("😮 YAWNING", yawn_msg))
            if head_tilted:
                state.alerts.append(("😴 HEAD DROOP", tilt_msg))
            if trend_fatigue:
                state.alerts.append(("⚠️ FATIGUE TREND", trend_msg))
            
            if state.alerts:
                self.last_alert_time = current_time
        
        # Calculate fatigue level (0-100%)
        fatigue_factors = []
        
        # Factor 1: EAR (lower = more fatigue)
        ear_factor = max(0, min(1, (0.35 - ear) / 0.2))  # Normalize around 0.35
        fatigue_factors.append(ear_factor)
        
        # Factor 2: MAR (higher = more fatigue from yawning)
        mar_factor = max(0, min(1, (mar - 0.4) / 0.3))  # Normalize around 0.4
        fatigue_factors.append(mar_factor * 0.7)  # Weight yawn less than eyes
        
        # Factor 3: Head tilt (extreme tilt = fatigue)
        tilt_factor = max(0, min(1, abs(head_tilt) / 30))
        fatigue_factors.append(tilt_factor * 0.5)  # Weight tilt less
        
        # Factor 4: Consecutive closed eyes
        if self.consecutive_closed_eyes > 0:
            closed_factor = min(1, self.consecutive_closed_eyes / 30)
            fatigue_factors.append(closed_factor)
        
        # Calculate overall fatigue level
        if fatigue_factors:
            state.fatigue_level = min(100, np.mean(fatigue_factors) * 100)
        
        # Final fatigue decision
        state.is_fatigued = (
            eye_closed or 
            (yawning and head_tilted) or 
            state.fatigue_level > 60
        )
        
        return state

# ---------- Load Model ----------
base_options = python.BaseOptions(model_asset_path='face_landmarker_v2_with_blendshapes.task')
options = vision.FaceLandmarkerOptions(
    base_options=base_options,
    running_mode=vision.RunningMode.VIDEO,
    num_faces=1,
    output_face_blendshapes=False,
    output_facial_transformation_matrixes=False
)
landmarker = vision.FaceLandmarker.create_from_options(options)

# Utility Functions
def calculate_angle(p1, p2):
    dx = p2[0] - p1[0]
    dy = p2[1] - p1[1]
    return np.degrees(np.arctan2(dy, dx))

def euclidean(p1, p2):
    return np.linalg.norm(np.array(p1) - np.array(p2))

def calculate_ear(eye_pts):
    p1, p2, p3, p4, p5, p6 = eye_pts
    return (euclidean(p2,p6) + euclidean(p3,p5)) / (2.0 * euclidean(p1,p4))

def calculate_mar(mouth_pts):
    p1, p2, p3, p4, p5, p6 = mouth_pts[:6]
    return (euclidean(p2,p6) + euclidean(p3,p5)) / (2.0 * euclidean(p1,p4))

# Landmark Indices
LEFT_EYE = [33, 160, 158, 133, 153, 144]
RIGHT_EYE = [362, 385, 387, 263, 373, 380]
MOUTH = [13, 14, 78, 308, 82, 312, 87, 317]

# Initialize detector
detector = SimpleFatigueDetector()

# Start video capture
cap = cv2.VideoCapture(0)
cv2.namedWindow("Fatigue Detection System", cv2.WINDOW_NORMAL)
global_start_time = time.time()

print("Simple Fatigue Detection System")
print("=" * 60)
print("LEFT SIDE: Normal Video with Fatigue Analytics")
print("RIGHT SIDE: Analyzed View with Face Mesh Detection")
print("=" * 60)
print(f"- EAR threshold: {detector.EAR_THRESHOLD} (Eye closure detection)")
print(f"- MAR threshold: {detector.MAR_THRESHOLD} (Yawning detection)")
print(f"- Head tilt threshold: {detector.HEAD_TILT_THRESHOLD}°")
print("-" * 60)
print("Press 'q' to quit")
print("=" * 60)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    frame = cv2.flip(frame, 1)
    
    # Create a copy for mesh visualization
    mesh_frame = frame.copy()
    
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_frame)
    timestamp = int((time.time() - global_start_time) * 1000)
    
    result = landmarker.detect_for_video(mp_image, timestamp)
    
    if result.face_landmarks:
        face_landmarks = result.face_landmarks[0]
        h, w, _ = frame.shape
        
        # Extract points
        nose = face_landmarks[1]
        left_eye_outer = face_landmarks[33]
        right_eye_outer = face_landmarks[263]
        chin = face_landmarks[152]
        
        nose_pt = (int(nose.x * w), int(nose.y * h))
        left_eye_pt = (int(left_eye_outer.x * w), int(left_eye_outer.y * h))
        right_eye_pt = (int(right_eye_outer.x * w), int(right_eye_outer.y * h))
        chin_pt = (int(chin.x * w), int(chin.y * h))
        
        # Calculate metrics
        roll_angle = calculate_angle(left_eye_pt, right_eye_pt)
        pitch_distance = chin_pt[1] - nose_pt[1]
        
        left_eye_pts = [(int(face_landmarks[i].x*w), int(face_landmarks[i].y*h)) for i in LEFT_EYE]
        right_eye_pts = [(int(face_landmarks[i].x*w), int(face_landmarks[i].y*h)) for i in RIGHT_EYE]
        mouth_pts = [(int(face_landmarks[i].x*w), int(face_landmarks[i].y*h)) for i in MOUTH]
        
        ear = (calculate_ear(left_eye_pts) + calculate_ear(right_eye_pts)) / 2.0
        mar = calculate_mar(mouth_pts)
        
        # Detect fatigue
        current_time = time.time()
        fatigue_state = detector.detect(ear, mar, roll_angle, current_time)
        
        # ========== LEFT SIDE: Normal Video with Analytics ==========
        normal_view = frame.copy()
        
        # Add header for left side
        cv2.putText(normal_view, "LIVE FEED - FATIGUE ANALYTICS", (30, 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        
        # Display metrics with descriptions
        y_offset = 80
        
        # EAR with description
        cv2.putText(normal_view, f"EAR (Eye Aspect Ratio): {ear:.3f}", (30, y_offset), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        cv2.putText(normal_view, f"  → Measures eye openness (lower = more closed)", (30, y_offset+20), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.4, (200, 200, 200), 1)
        
        # MAR with description
        cv2.putText(normal_view, f"MAR (Mouth Aspect Ratio): {mar:.3f}", (30, y_offset+50), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        cv2.putText(normal_view, f"  → Measures mouth openness (higher = yawning)", (30, y_offset+70), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.4, (200, 200, 200), 1)
        
        # Head tilt with description
        cv2.putText(normal_view, f"Head Tilt: {roll_angle:.1f}°", (30, y_offset+100), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        cv2.putText(normal_view, f"  → Head angle (higher = drooping)", (30, y_offset+120), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.4, (200, 200, 200), 1)
        
        # Fatigue level bar with description
        bar_length = 250
        filled_length = int(bar_length * fatigue_state.fatigue_level / 100)
        bar_color = (0, 255, 0) if fatigue_state.fatigue_level < 40 else (0, 255, 255) if fatigue_state.fatigue_level < 70 else (0, 0, 255)
        
        cv2.putText(normal_view, "Fatigue Level:", (30, y_offset+160), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        cv2.rectangle(normal_view, (30, y_offset+170), (30+filled_length, y_offset+190), bar_color, -1)
        cv2.rectangle(normal_view, (30, y_offset+170), (30+bar_length, y_offset+190), (255, 255, 255), 2)
        cv2.putText(normal_view, f"{fatigue_state.fatigue_level:.0f}%", (30+bar_length+10, y_offset+185), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        
        # Status indicators with descriptions
        status_y = y_offset + 220
        
        # Eye status
        eye_status = "CLOSED" if ear < detector.EAR_THRESHOLD else "OPEN"
        eye_color = (0, 0, 255) if ear < detector.EAR_THRESHOLD else (0, 255, 0)
        cv2.putText(normal_view, f"Eyes: {eye_status}", (30, status_y), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, eye_color, 2)
        
        # Mouth status
        mouth_status = "YAWNING" if mar > detector.MAR_THRESHOLD else "NORMAL"
        mouth_color = (0, 0, 255) if mar > detector.MAR_THRESHOLD else (0, 255, 0)
        cv2.putText(normal_view, f"Mouth: {mouth_status}", (30, status_y+30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, mouth_color, 2)
        
        # Head status
        head_status = "TILTED" if abs(roll_angle) > detector.HEAD_TILT_THRESHOLD else "NORMAL"
        head_color = (0, 0, 255) if abs(roll_angle) > detector.HEAD_TILT_THRESHOLD else (0, 255, 0)
        cv2.putText(normal_view, f"Head: {head_status}", (30, status_y+60), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, head_color, 2)
        
        # Alerts on left side
        if fatigue_state.alerts:
            alert_y = status_y + 100
            cv2.putText(normal_view, "ACTIVE ALERTS:", (30, alert_y), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            for i, (alert_type, msg) in enumerate(fatigue_state.alerts):
                cv2.putText(normal_view, f"{alert_type}", (30, alert_y+30 + i*25), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
                cv2.putText(normal_view, f"  {msg}", (30, alert_y+50 + i*25), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.4, (200, 200, 200), 1)
        
        # ========== RIGHT SIDE: Mesh View with Detection Visualization ==========
        mesh_view = mesh_frame
        
        # Add header for right side
        cv2.putText(mesh_view, "FACE MESH ANALYSIS", (30, 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
        
        # Draw eye landmarks with color coding
        for pt in left_eye_pts + right_eye_pts:
            color = (0, 255, 0) if ear > detector.EAR_THRESHOLD else (0, 0, 255)
            cv2.circle(mesh_view, pt, 3, color, -1)
            # Add small label
            if pt == left_eye_pts[0] or pt == right_eye_pts[0]:
                cv2.putText(mesh_view, "Eye", (pt[0]+5, pt[1]-5), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.3, color, 1)
        
        # Draw mouth landmarks with color coding
        for pt in mouth_pts:
            color = (0, 255, 0) if mar < detector.MAR_THRESHOLD else (0, 0, 255)
            cv2.circle(mesh_view, pt, 3, color, -1)
            if pt == mouth_pts[0]:
                cv2.putText(mesh_view, "Mouth", (pt[0]+5, pt[1]-5), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.3, color, 1)
        
        # Draw head pose lines with descriptions
        cv2.line(mesh_view, left_eye_pt, right_eye_pt, (255, 0, 0), 2)
        cv2.putText(mesh_view, "Eye Line (Tilt)", 
                   ((left_eye_pt[0] + right_eye_pt[0])//2 - 50, (left_eye_pt[1] + right_eye_pt[1])//2 - 10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 0, 0), 1)
        
        cv2.line(mesh_view, nose_pt, chin_pt, (255, 255, 0), 2)
        cv2.putText(mesh_view, "Nose-Chin (Pitch)", 
                   ((nose_pt[0] + chin_pt[0])//2 - 50, (nose_pt[1] + chin_pt[1])//2), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 0), 1)
        
        # Add measurement labels on mesh view
        cv2.putText(mesh_view, f"EAR: {ear:.3f}", (w-200, 80), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        cv2.putText(mesh_view, f"MAR: {mar:.3f}", (w-200, 110), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        cv2.putText(mesh_view, f"Tilt: {roll_angle:.1f}°", (w-200, 140), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        # Legend for colors
        cv2.putText(mesh_view, "Green: Normal", (w-200, h-90), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 1)
        cv2.putText(mesh_view, "Red: Fatigue Indicator", (w-200, h-70), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 255), 1)
        cv2.putText(mesh_view, "Blue Lines: Measurements", (w-200, h-50), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 0, 0), 1)
        
        # Overall fatigue status on mesh view
        if fatigue_state.is_fatigued:
            # Create a translucent overlay for warning
            overlay = mesh_view.copy()
            cv2.rectangle(overlay, (w//2-150, 50), (w//2+150, 100), (0, 0, 255), -1)
            cv2.addWeighted(overlay, 0.3, mesh_view, 0.7, 0, mesh_view)
            cv2.putText(mesh_view, "FATIGUE DETECTED", (w//2-140, 80), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
    
    else:
        # No face detected
        normal_view = frame.copy()
        mesh_view = frame.copy()
        cv2.putText(normal_view, "NO FACE DETECTED", (30, h//2), 
                   cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 2)
        cv2.putText(mesh_view, "NO FACE DETECTED", (30, h//2), 
                   cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 2)
    
    # Combine both views side by side
    # Make sure both have same height, if not resize
    if normal_view.shape[0] != mesh_view.shape[0]:
        h = min(normal_view.shape[0], mesh_view.shape[0])
        normal_view = cv2.resize(normal_view, (int(normal_view.shape[1] * h / normal_view.shape[0]), h))
        mesh_view = cv2.resize(mesh_view, (int(mesh_view.shape[1] * h / mesh_view.shape[0]), h))
    
    combined = np.hstack((normal_view, mesh_view))
    
    # Add a vertical divider line
    divider_x = normal_view.shape[1]
    cv2.line(combined, (divider_x, 0), (divider_x, combined.shape[0]), (255, 255, 255), 2)
    
    # Add labels for each side
    cv2.putText(combined, "NORMAL VIEW", (30, 60), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
    cv2.putText(combined, "ANALYZED VIEW", (divider_x + 30, 60), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
    
    # Show the combined view
    cv2.imshow("Fatigue Detection System", combined)
    
    # Exit on 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

W0000 00:00:1771092061.657271   57853 face_landmarker_graph.cc:174] Sets FaceBlendshapesGraph acceleration to xnnpack by default.
W0000 00:00:1771092061.664285   57859 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1771092061.683108   57865 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Simple Fatigue Detection System
LEFT SIDE: Normal Video with Fatigue Analytics
RIGHT SIDE: Analyzed View with Face Mesh Detection
- EAR threshold: 0.25 (Eye closure detection)
- MAR threshold: 0.6 (Yawning detection)
- Head tilt threshold: 15°
------------------------------------------------------------
Press 'q' to quit
